Bias Field Correction

BFC is done to increase homogenity in intensity around the main head region.

In [2]:
%matplotlib inline
import os
from helpers import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.5.4
SimpleITK version = 2.5.0


In [3]:
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
print(f'project folder = {BASE_DIR}')

project folder = c:\Users\CHEEZYJEEZY\Desktop\MRIdata_preprocessing


In [4]:
raw_examples = [
    'fsl-open-dev_sub-001_T1w.nii.gz',
    'wash-120_sub-001_T1w.nii.gz',
    'kf-panda_sub-01_ses-3T_T1w.nii.gz',
    'listen-task_sub-UTS01_ses-1_T1w.nii.gz'
]

In [5]:
raw_img_path = os.path.join(BASE_DIR,'assets', 'raw_examples', raw_examples[0])

In [6]:
raw_img_sitk = sitk.ReadImage(raw_img_path, sitk.sitkFloat32)
raw_img_sitk = sitk.DICOMOrient(raw_img_sitk, 'RPS')
raw_img_sitk_arr = sitk.GetArrayFromImage(raw_img_sitk)
print(raw_img_sitk_arr.shape)
explore_3D_array(arr = raw_img_sitk_arr,
                 cmap = 'nipy_spectral')

(192, 192, 160)


interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

Create head mask using threshold

In [7]:
transformed = sitk.RescaleIntensity(raw_img_sitk, 0, 255)

transformed = sitk.LiThreshold(transformed, 0,1)

head_mask = transformed

explore_3D_array_comparison(
    arr_before = sitk.GetArrayFromImage(raw_img_sitk),
    arr_after = sitk.GetArrayFromImage(head_mask),
)

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

Bias Correction

In [8]:
shrinkFactor = 4
inputImage = raw_img_sitk

inputImage = sitk.Shrink( raw_img_sitk, [ shrinkFactor ] * inputImage.GetDimension())
maskImage = sitk.Shrink( head_mask, [shrinkFactor]* inputImage.GetDimension())

bias_corrector = sitk.N4BiasFieldCorrectionImageFilter()

corrected = bias_corrector.Execute(inputImage, maskImage)

In [9]:
log_bias_field = bias_corrector.GetLogBiasFieldAsImage(raw_img_sitk)
corrected_image_full_resolution = raw_img_sitk / sitk.Exp(log_bias_field)

explore_3D_array_comparison(
    sitk.GetArrayFromImage(raw_img_sitk),
    sitk.GetArrayFromImage(corrected_image_full_resolution),
    cmap = 'viridis',
)

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

Inspect the bias field

In [10]:
temp = sitk.Exp(log_bias_field)
temp = sitk.Mask(temp, head_mask)
explore_3D_array(sitk.GetArrayFromImage(temp))

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

Save the Image

In [13]:
raw_example = raw_examples[0]

In [15]:
out_folder = os.path.join(BASE_DIR, 'assets', 'preprocessed')
out_folder = os.path.join(out_folder, raw_example.split('.')[0])
os.makedirs(out_folder, exist_ok = True)

out_filename = add_suffix_to_filename(raw_example, suffix = 'biasFieldCorrected')
out_path = os.path.join(out_folder, out_filename)

print(raw_img_path[len(BASE_DIR):])
print(out_path[len(BASE_DIR):])

\assets\raw_examples\fsl-open-dev_sub-001_T1w.nii.gz
\assets\preprocessed\fsl-open-dev_sub-001_T1w\fsl-open-dev_sub-001_T1w_biasFieldCorrected.nii.gz
